In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import modules.model as Model
import pandas as pd

# Train test split
* Phần này tập trung chủ yếu vào việc chuẩn bị dữ liệu trước khi tiến hành đào tạo hàng loạt các model.

Đọc dữ liệu

In [3]:
data = pd.read_csv("./data/normalize_reviews.csv").fillna("")[['raw_comment', 'normalize_comment', 'label']]

data.head()

,raw_comment,normalize_comment,label
0,Giao hàng kh đúng cần phê bình hjjjjjhhd...,giao hàng không đúng cần phê bình,0
1,Chất lượng sản phẩm tạm được. Giao...,chất lượng sản phẩm tạm được giao ...,0
2,Ko có lắc tay như hình,không có lắc tay như hình,0
3,Giao hàng lâu. Bảo có lắc tay mà k thâ...,giao hàng lâu bảo có lắc tay mà không ...,0
4,"Mình mua 2 cái, một dùng ok. Một cái k...",mua cái một dùng ok một cái không chạ...,0


Do **sklearn** hoạt động tốt nhất trên text data có dạng mã hóa là **NFC**, nên trước tiên ta sẽ chuyển toàn bộ các feature có `dtype` là `str` được biểu diễn dưới dạng mã hóa NFC.

In [4]:
data = Model.textNFCformat(data, ['raw_comment', 'normalize_comment'])

data.head()

,raw_comment,normalize_comment,label
0,Giao hàng kh đúng cần phê bình hjjjjjhhdjdjdjd...,giao hàng không đúng cần phê bình,0
1,Chất lượng sản phẩm tạm được. Giao sai 1 món. ...,chất lượng sản phẩm tạm được giao sai món đóng...,0
2,Ko có lắc tay như hình,không có lắc tay như hình,0
3,Giao hàng lâu. Bảo có lắc tay mà k thấy giao. ...,giao hàng lâu bảo có lắc tay mà không thấy gia...,0
4,"Mình mua 2 cái, một dùng ok. Một cái không chạ...",mua cái một dùng ok một cái không chạy được,0


Các vectorizer object của **sklearn** mặc định chúng sẽ xóa toàn bộ các **punctuation** [kí tự đặc biệt] trong input truyền vào. Như vậy các emoji của ta sẽ bị xóa toàn bộ khi transform vectorizing. Như vậy, ta sẽ không lưu chúng dưới dạng các punctuation mà dùng decode của chúng - sau đó ta tạo feature `emoji_decode` để lưu chúng.  

In [5]:
data['emoji_decode'] = data['raw_comment'].apply(lambda s: Model.expandEmojisDecode(s))
data = data[['raw_comment', 'normalize_comment', 'emoji_decode', 'label']]

data.head()

,raw_comment,normalize_comment,emoji_decode,label
0,Giao hàng kh đúng cần phê bình hjjjjjhhdjdjdjd...,giao hàng không đúng cần phê bình,,0
1,Chất lượng sản phẩm tạm được. Giao sai 1 món. ...,chất lượng sản phẩm tạm được giao sai món đóng...,,0
2,Ko có lắc tay như hình,không có lắc tay như hình,,0
3,Giao hàng lâu. Bảo có lắc tay mà k thấy giao. ...,giao hàng lâu bảo có lắc tay mà không thấy gia...,,0
4,"Mình mua 2 cái, một dùng ok. Một cái không chạ...",mua cái một dùng ok một cái không chạy được,cry,0


Tiến hành chọn các mẫu có feature `emoji_decode` không phải là chuổi rỗng và lưu vào biến `emoji_data`.

In [6]:
emoji_data = data[data['emoji_decode'] != ""].reset_index(drop=True)

emoji_data.head()

,raw_comment,normalize_comment,emoji_decode,label
0,"Mình mua 2 cái, một dùng ok. Một cái không chạ...",mua cái một dùng ok một cái không chạy được,cry,0
1,Giao sai màu sai size có 1 dép lông size 36 à ...,giao sai màu sai size có dép lông size à còn đ...,cursing_face cursing_face cursing_face,0
2,Đơn hàng đã thanh toán airpay rồi mà shipper g...,đơn hàng thanh toán mà giao cho người lạ lỡ đơ...,roll_eyes,0
3,"Áo croptop freesize rộng với mình \nMình m62, ...",áo rộng mình kg,thumbsup thumbsup thumbsup thumbsup thumbsup t...,0
4,Khá buồn . Đặt 2 cái đều không chạy giờ . 1 cá...,khá buồn đặt cái đều không chạy giờ cái sai mẫ...,shrug female_sign,0


Giờ thì ta sẽ tiến hành chia `emoji_data` thành training data và test data với size của test data là 15%, sau đó ta lưu chúng dưới dạng file **.csv**.

In [7]:
Model.dataSplitSaved(emoji_data, 0.15, "./data/emoji_data")

📢 Your dataset has saved at ./data/emoji_data.


Ta cũng sẽ tiến hành chia training data và test data cho 

In [10]:
data.head()

,raw_comment,normalize_comment,emoji_decode,label
0,Giao hàng kh đúng cần phê bình hjjjjjhhdjdjdjd...,giao hàng không đúng cần phê bình,,0
1,Chất lượng sản phẩm tạm được. Giao sai 1 món. ...,chất lượng sản phẩm tạm được giao sai món đóng...,,0
2,Ko có lắc tay như hình,không có lắc tay như hình,,0
3,Giao hàng lâu. Bảo có lắc tay mà k thấy giao. ...,giao hàng lâu bảo có lắc tay mà không thấy gia...,,0
4,"Mình mua 2 cái, một dùng ok. Một cái không chạ...",mua cái một dùng ok một cái không chạy được,cry,0


In [11]:
Model.dataSplitSaved(data, 0.15, "./data/data")

📢 Your dataset has saved at ./data/data.
